<a href="https://colab.research.google.com/github/Putt2020/pytorch_beginner/blob/main/pytorch_CNN_beginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [3]:
transform = transforms.ToTensor()

In [5]:
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
print(train_data)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()


In [8]:
# Create data loader with batch size of 32
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [9]:
#Define CNN Model

"""
in_channels=1, input -> WxH image with 1 channel (grayscale)
out_channels=10, -> Amount of feature layers we want to create (will get 10xW'xH')
kernel_size=3, -> size of the filter in this case it is 3x3
stride=1, -> how much the filter moves in this case is 1 pixel at a time
padding=1 -> pad the border of the image with 0 i.e. from 10x10 to 12x12
"""
conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, stride=1, padding=1)
conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3, stride=1, padding=1)

In [10]:
for i, (X_train, y_train) in enumerate(train_loader):
    print(i, X_train.shape, y_train.shape)
    break

0 torch.Size([32, 1, 28, 28]) torch.Size([32])


In [11]:
# Create CNN model class
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, stride=1, padding=1) #from (32,1,28,28) to (32,10,28,28) becasue kernel = -2 and padding = +2
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) #from (32,10,28,28) to (32,10,14,14)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3, stride=1, padding=1) #from (32,10,14,14) to (32,20,14,14)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) #from (32,10,14,14) to (32,10,7,7)
        self.fc1 = nn.Linear(in_features=20*7*7, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)

    def forward(self):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        # Use view to flattern data
        x = x.view(-1, 20*7*7)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)


In [13]:
#Create an instance of model
torch.manual_seed(42)
model = CNN_Model()
model

CNN_Model(
  (conv1): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=980, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [14]:
#Optimzer and loss
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)